# **Exploración Preliminar archivos Yelp**

El objetivo de este notebook es poder entender la naturaleza, formato y estructuras que contienen los archivos de Yelp, de tal modo que nos permita posteriormente realizar un proceso de ETL para preparar el datawarehouse. El análisis se realizo en los siguientes archivos:
- business.pkl
- review.json
- user.parquet
- checkin.json
- tip.json

## **Librerias utlizadas**

In [1]:
#AÑADIR CADA NUEVA LIBRERIA QUE SE UTILICE
import pandas as pd
import numpy as np
import requests


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Business**

## **2. Review**

## **3. User**

## **4. Checkin**

In [12]:
# Leemos el documento "chekin" en formato json.
chekin = pd.read_json('/content/drive/MyDrive/Final Project /Yelp/checkin.json', lines=True)

# Observamos el dataframe "chekin" con las columnas y los datos que contienen.
chekin.sample(5)

,business_id,date
113552,r9h2tC9eSpgWAbp7hw1QvA,"2017-01-16 17:27:50, 2017-01-20 20:18:27, 2017..."
113979,rMqWPcEwfnUZ-f8coYHEsA,2012-05-18 14:09:04
88788,f8gfe7S-T15ll2sMURVeqg,"2016-08-03 15:04:48, 2016-11-15 20:04:44, 2016..."
65224,UhA77Rfgcm46B7pUDCVh4g,"2020-08-31 21:03:38, 2020-08-31 23:22:32, 2020..."
86501,e2xecLS4kmy87QdA0QI-Ew,2016-06-09 00:21:48


In [13]:
# Obtenemos la información del dataframe como los nombres de las columnas, los valores totales, si cuentan con nulos y los tipos de datos.
chekin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


##**Comentarios:**
El dataframe cuenta con dos columnas, "business_id" y "date". Las fechas están conjuntas separadas en comas en formato object. Por lo que se realizará un EDA en donde podremos observer el ID con más fechas separadas y el ID con fecha más antigua y con la fecha más nueva.

4.1. ID con más fechas separadas

In [14]:
# Creamos una función para contar las fechas de la columna "date".
def contar_fechas(fila):
    # Dividimos la cadena en fechas usando la coma como separador y contamos los elementos.
    fechas = fila.split(', ')
    return len(fechas)

# Aplicamos la función a la columna "date" y creamos una nueva columna 'cantidad_fechas'
chekin['cantidad_fechas'] = chekin['date'].apply(contar_fechas)

# Observamos los cambios en el dataframe.
chekin.sample(5)

,business_id,date,cantidad_fechas
80759,bGPP22tfSHkc5OkYARGZBA,"2010-06-12 17:31:49, 2010-06-12 21:32:41, 2011...",120
45384,L1HNEw8NOWGGH3Uyttns4A,"2010-11-27 02:51:15, 2011-04-23 21:50:25, 2011...",47
124505,wRMdz34j57AUAQc28v9-hQ,"2010-07-10 01:14:53, 2010-08-08 00:50:41, 2010...",1855
114901,rqD6cyApke-O-SkppUogTg,"2010-01-23 15:58:06, 2010-03-27 14:46:02, 2011...",39
106461,nl33PVk5w9JIqOlEGPS_ZA,"2011-01-09 16:09:26, 2011-04-02 01:56:52, 2011...",26


In [15]:
# Encontramos el ID con la cantidad máxima de fechas.
id_max_fechas = chekin.loc[chekin['cantidad_fechas'].idxmax()]['business_id']

# Observamos el id que tiene la cantidad máxima de fechas.
id_max_fechas

'-QI8Qi8XWH3D8y8ethnajA'

4.2. ID's con fechas antiguas y fechas recientes

In [16]:
# Dividimos las fechas en una columna por comas.
chekin['fechas'] = chekin['date'].str.split(',')

# Duplicamos las filas para cada fecha con su respectivo ID.
chekin=chekin.explode('fechas').reset_index(drop=True)

# Eliminamos la columna original.
chekin.drop('date',axis=1,inplace=True)

# Observamos los cambios en el dataframe.
chekin.head(5)

,business_id,cantidad_fechas,fechas
0,---kPU91CF4Lq2-WlRu9Lw,11,2020-03-13 21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,11,2020-06-02 22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,11,2020-07-24 22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,11,2020-10-24 21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,11,2020-12-09 21:23:33


In [17]:
# Buscamos el ID con la fecha más antigua.
fecha_minima = chekin['fechas'].min()
id_fecha_minima = chekin.loc[chekin['fechas'] == fecha_minima]
id_fecha_minima

,business_id,cantidad_fechas,fechas
8511938,cFk2SMsJ-2cmXKLFFkP9JA,587,2010-01-17 02:15:25


In [18]:
# Buscamos el ID con la fecha más reciente.
fecha_maxima = chekin['fechas'].max()
id_fecha_maxima = chekin.loc[chekin['fechas'] == fecha_maxima]
id_fecha_maxima

,business_id,cantidad_fechas,fechas
3188598,DjpCcVUuolhT6FF78W1aQw,1,2022-01-19 01:15:21


In [19]:
# Transformamos la columna "fechas" en el formato datetime "aaaa-mm-dd".
chekin['fechas'] = pd.to_datetime(chekin['fechas'],  errors='coerce').dt.strftime('%Y-%m-%d')
chekin.sample(5)

,business_id,cantidad_fechas,fechas
7352668,Xdy7tkuV0ViNFVxcsL4wjA,126,2021-06-05
310509,06pNHtxgPhGhZpM0Z2uzNw,2364,2016-10-14
548544,1KpNlu8VM8ukw3uOzmMFPA,1042,2013-05-12
8158360,abvsCLYEOvhsWSlgiTvbCQ,90,2018-12-19
11385795,q6hWVlVeiCgMgM22wt1wfw,2326,2015-04-18


## **5. Tip**